In [4]:
date = "201911"

StatementMeta(, , , Waiting, )

In [5]:
fhvTaxisFilePath = "abfss://taxidata@pstaxisdatalake.dfs.core.windows.net/FhvTaxis_" + date + ".csv"

StatementMeta(PSSparkPool, 5, 5, Finished, Available)

In [6]:
fhvTaxiTripDataDF = (
                        spark
                            .read
                            .option("header", "true")
                            .option("inferSchema", "true")
                            .csv(fhvTaxisFilePath)
                    )

StatementMeta(PSSparkPool, 5, 6, Finished, Available)

In [7]:
from pyspark.sql.functions import *

fhvTaxiTripDataDF = (
                        fhvTaxiTripDataDF

                            # Select limited columns
                            .select(
                                    "hvfhs_license_num",
                                    "dispatching_base_num",
                                    "Pickup_DateTime", 
                                    "DropOff_DateTime", 
                                    "PUlocationID", 
                                    "DOlocationID"                                    
                                )

                            #Rename the columns
                            .withColumnRenamed("hvfhs_license_num", "CompanyLicenseId")
                            .withColumnRenamed("dispatching_base_num", "BaseLicenseId")
                            .withColumnRenamed("Pickup_DateTime", "PickupTime")
                            .withColumnRenamed("DropOff_DateTime", "DropTime")
                            .withColumnRenamed("PUlocationID", "PickupLocationId")
                            .withColumnRenamed("DOlocationID", "DropLocationId")

                            # Create derived columns for year, month and day
                            .withColumn("TripYear", year(col("PickupTime")))
                            .withColumn("TripMonth", month(col("PickupTime")))
                            .withColumn("TripDay", dayofmonth(col("PickupTime")))

                            # Filter records based on PickupTime
                            .where("PickupTime >= '2019-11-01' AND PickupTime < '2019-12-01'")
                    )

StatementMeta(PSSparkPool, 5, 7, Finished, Available)

In [8]:
(
    fhvTaxiTripDataDF
        .write
        .partitionBy("TripYear", "TripMonth", "TripDay")
        .mode("overwrite")
        .option("path", "abfss://taxioutput@pstaxisdatalake.dfs.core.windows.net/Facts/FhvTaxis.parquet")
        .saveAsTable("FhvWarehouse.FHVTrips")
)

StatementMeta(PSSparkPool, 5, 8, Finished, Available)